In [1]:
import os
os.chdir("/tf/src/examples/greetings")

from greetings import Dataset
import tensorflow as tf
import numpy as np
import logging
from greetings import GreetingModel
from greetings import cfg
logging.basicConfig(
        level=cfg.LOG_LVL,
        filename=cfg.LOG_FILENAME,
        format='%(message)s')

In [2]:
data = Dataset()

In [3]:
model = GreetingModel()

In [4]:
RNG_SEED = 11
to_gpu = tf.data.experimental.copy_to_device("/gpu:0")
train = data.train.shuffle(
    buffer_size=10000, seed=RNG_SEED, reshuffle_each_iteration=False)\
    .batch(512).apply(to_gpu)
val = data.test.batch(512).apply(to_gpu)
with tf.device("/gpu:0"):
    train = train.prefetch(3)
    val = val.prefetch(3)

In [5]:
X, y = next(val.as_numpy_iterator())
print("source idx from source vocab\n", X[0][:3])
print("\nsource idx from target vocab\n", X[1][:3])
print("\ntarget idx from target vocab\n", y[0][:3])
print("\ntarget idx from source vocab\n", y[1][:3])

source idx from source vocab
 [[   2   24    6    5    1    1    3    0    0    0]
 [   2   22 2239    3    0    0    0    0    0    0]
 [   2   26   27    6    5 2042    4    3    0    0]]

source idx from target vocab
 [[2 1 1 1 1 1 3 0 0 0]
 [2 1 1 3 0 0 0 0 0 0]
 [2 1 1 1 1 1 1 3 0 0]]

target idx from target vocab
 [[2 4 5 6 7 8 1 1 9 3]
 [2 4 5 6 7 8 1 9 3 0]
 [2 4 5 6 7 8 1 9 3 0]]

target idx from source vocab
 [[   2    1    1    1    1    8    1    1    1    3]
 [   2    1    1    1    1    8 2239    1    3    0]
 [   2    1    1    1    1    8 2042    1    3    0]]


In [6]:
model.fit(train, cfg.EPOCHS, cfg.MODEL_SAVE, val)

Epoch 1/5: 10it [00:29,  2.94s/it, train-loss=9.5871, train-bleu=0.685, train-bleu-smooth=0.685, val-loss=6.8009, val-bleu=0.813, val-bleu-smooth=0.813]
Epoch 2/5: 10it [00:11,  1.17s/it, train-loss=0.9940, train-bleu=0.917, train-bleu-smooth=0.917, val-loss=7.7019, val-bleu=0.85, val-bleu-smooth=0.85]
Epoch 3/5: 10it [00:11,  1.15s/it, train-loss=0.2673, train-bleu=0.984, train-bleu-smooth=0.984, val-loss=10.9774, val-bleu=0.94, val-bleu-smooth=0.94]
Epoch 4/5: 10it [00:11,  1.17s/it, train-loss=0.0676, train-bleu=0.996, train-bleu-smooth=0.996, val-loss=13.2462, val-bleu=0.932, val-bleu-smooth=0.932]
Epoch 5/5: 10it [00:11,  1.18s/it, train-loss=0.0340, train-bleu=0.998, train-bleu-smooth=0.998, val-loss=13.6505, val-bleu=0.928, val-bleu-smooth=0.928]


In [10]:
def idx2str(pred_y, X):
    ret = []
    vocab_len = model.vocab.get_vocab_size("target")
    for idx in pred_y:
        if idx < vocab_len:
            ret.append(model.vocab.get_token_text(idx, "target"))
        else:
            ret.append(model.vocab.get_token_text(X[idx-vocab_len], "source"))
    return ret

In [11]:
pred, pred_proba = model.predict(train)
for i, Xy in enumerate(train.unbatch().take(10)):
    X, y = Xy
    s = model.vocab.inverse_transform(X[0].numpy()[np.newaxis, :], "source")[0]
    t = model.vocab.inverse_transform(y[0].numpy()[np.newaxis, :], "target")[0]
    print(f"Source: {' '.join(s)}\nTarget: {' '.join(t)}\n")
    for j in range(10):
        p = idx2str(pred[i][j].numpy(), X[0].numpy())
        print(f"Predicted: {' '.join(p)}\tProba: {tf.exp(pred_proba[i][j])}")
    print("")

Source: <S> Hi there . It 's Nyla Wais . EOS
Target: <S> Nice to meet you , UNKNOWN UNKNOWN ! EOS

Predicted: Nice to meet you , Nyla Wais ! EOS	Proba: 0.9990193843841553
Predicted: Nice meet you , Nyla Wais ! EOS EOS	Proba: 0.0003403811133466661
Predicted: Nice to meet , Nyla Wais ! EOS EOS	Proba: 0.00019790069200098515
Predicted: Nice to meet you Nyla Wais ! EOS EOS	Proba: 0.0001568654115544632
Predicted: to meet you , Nyla Wais ! EOS EOS	Proba: 7.89705227361992e-05
Predicted: Nice to meet you , Nyla Wais Wais !	Proba: 6.864441820653155e-05
Predicted: Nice to you , Nyla Wais ! EOS EOS	Proba: 3.161652057315223e-05
Predicted: Nice to meet meet you , Nyla Wais !	Proba: 2.3224441974889487e-05
Predicted: Nice to meet you you , Nyla Wais !	Proba: 1.4644455404777545e-05
Predicted: Nice to meet you , Nyla Wais . !	Proba: 1.429650183126796e-05

Source: <S> Its Roseann , actually . EOS PAD PAD PAD
Target: <S> Nice to meet you , Roseann ! EOS PAD

Predicted: Nice to meet you , Roseann ! EOS EOS

In [12]:
pred, pred_proba = model.predict(val.unbatch().take(50).batch(5))
for i, Xy in enumerate(val.unbatch().take(10)):
    X, y = Xy
    s = model.vocab.inverse_transform(X[0].numpy()[np.newaxis, :], "source")[0]
    t = model.vocab.inverse_transform(y[0].numpy()[np.newaxis, :], "target")[0]
    print(f"Source: {' '.join(s)}\nTarget: {' '.join(t)}\n")
    for j in range(10):
        p = idx2str(pred[i][j].numpy(), X[0].numpy())
        print(f"Predicted: {' '.join(p)}\tProba: {tf.exp(pred_proba[i][j])}")
    print("")

Source: <S> Please call me UNKNOWN UNKNOWN EOS PAD PAD PAD
Target: <S> Nice to meet you , UNKNOWN UNKNOWN ! EOS

Predicted: Nice to meet you , UNKNOWN UNKNOWN UNKNOWN !	Proba: 0.22575844824314117
Predicted: Nice to meet you , UNKNOWN UNKNOWN EOS EOS	Proba: 0.22129006683826447
Predicted: Nice to meet you , UNKNOWN UNKNOWN ! EOS	Proba: 0.19418850541114807
Predicted: Nice to meet you , UNKNOWN UNKNOWN UNKNOWN EOS	Proba: 0.11693324893712997
Predicted: Nice to meet you , UNKNOWN UNKNOWN UNKNOWN UNKNOWN	Proba: 0.0252889022231102
Predicted: Nice to meet you , UNKNOWN UNKNOWN UNKNOWN you	Proba: 0.023651210591197014
Predicted: Nice to meet you , UNKNOWN UNKNOWN UNKNOWN Nice	Proba: 0.015142795629799366
Predicted: Nice to meet you , UNKNOWN UNKNOWN you ,	Proba: 0.011790573596954346
Predicted: Nice to meet you , UNKNOWN UNKNOWN UNKNOWN meet	Proba: 0.011479678563773632
Predicted: Nice to meet you , UNKNOWN UNKNOWN , UNKNOWN	Proba: 0.007812898606061935

Source: <S> Its Albert EOS PAD PAD PAD PAD PAD